01. Initializations

In [1]:
import pandas as pd
import sys
import json
from nfstream import NFStreamer
import os
import datetime


root_folder = os.path.join("/", "RanD", "CREMEv2_Result", "20230310", "logs_working")
scenario = ["mirai", "disk_wipe", "ransomware", "resource_hijacking", "end_point_dos"]
scenario_folder = ["01_mirai", "02_disk_wipe", "03_ransomware", "04_resource_hijacking", "05_end_point_dos"]
syslog_folder = "syslog"
label_folder = "label_syslog"
filename = "dataset_generation.log"
label = {'mirai': [1, 2, 3, 5, 9, 11, 12, 13],
         'disk_wipe': [1, 2, 4, 6, 8, 14],
         'ransomware': [1, 2, 4, 6, 7, 12, 15],
         'resource_hijacking': [1, 2, 4, 6, 8, 12, 16],
         'end_point_dos': [1, 2, 4, 8, 10, 12, 17]
         }

ips = {"controller": ["192.168.56.111"],
       "data-logger": ["192.168.56.121"],
       "attacker": ["192.168.56.131"],
       "non-vulnerable": ["192.168.56.141", "192.168.56.142"],
       "vulnerable": ["192.168.56.151"],
       "malicious": ["192.168.56.161"],
       "benign": ["192.168.56.171"],
       "target": ["192.168.56.181"]
       }

2. Repairing the label

In [2]:
df = pd.read_csv(os.path.join(root_folder, label_folder, "original_label_syslog.csv"))

result_abs_path = os.path.join(root_folder, label_folder, "modified_label")
# print(ips["benign"][0])
df.Label[df.HostName=="192.168.56.171"] = 0
df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0
# print(df)

if not os.path.exists(result_abs_path):
    os.makedirs(result_abs_path)
df.to_csv(os.path.join(result_abs_path, "original_relabel_syslog.csv"), index=False)

/tmp/ipykernel_802176/2613811198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label[df.HostName=="192.168.56.171"] = 0
/tmp/ipykernel_802176/2613811198.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0
